# API Gateway Demo

In [2]:
api_url = "https://h31b7qwhgi.execute-api.us-east-1.amazonaws.com/dev/"
job_name = "job_001"

## Create Job

In [4]:
import json

import requests


def create_dummy_jobs(api_url: str, job_name: str, work_count: int):

    # Construct the full URL
    api_url = api_url.rstrip("/")
    endpoint = f"{api_url}/create_job"

    # Headers
    headers = {"Content-Type": "application/json"}

    works = [{"work_id": f"{job_name}_work_{i}", "s3_uris": ["s3://bucket/file1.txt"]} for i in range(work_count)]
    request_body = {"job_name": job_name, "works": works}

    try:
        # Make the POST request
        response = requests.post(endpoint, data=json.dumps(request_body), headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            print("API Response:", data)
        else:
            print(f"Error: API request failed with status code {response.status_code}")
            print("Response:", response.text)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


create_dummy_jobs(
    api_url=api_url,
    job_name=job_name,
    work_count=20,
)

API Response: {'message': 'Success'}


## Check Job Progress

In [5]:
import logging

import requests


def get_job_progress(api_url: str, job_name: str):
    """
    Query the job_progress endpoint with the given job_name.

    Args:
    api_url (str): The base URL of your API Gateway
    job_name (str): The name of the job to query

    Returns:
    dict: The JSON response from the API, or None if an error occurred
    """
    api_url = api_url.rstrip("/")
    # Construct the full URL
    endpoint = f"{api_url}/job_progress"

    # Set up the query parameters
    params = {"job_name": job_name}

    try:
        # Make the GET request
        response = requests.get(endpoint, params=params)

        # Check the status code
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            logging.info(f"No data found for job_name: {job_name}")
            return response.json()
        else:
            response.raise_for_status()

    except requests.RequestException as e:
        logging.exception(f"An error occurred: {e}")


progress = get_job_progress(
    api_url=api_url,
    job_name=job_name,
)
print(progress.keys())
print(progress)

dict_keys(['IN_QUEUE'])
{'IN_QUEUE': ['job_004_work_0', 'job_004_work_1', 'job_004_work_10', 'job_004_work_11', 'job_004_work_12', 'job_004_work_13', 'job_004_work_14', 'job_004_work_15', 'job_004_work_16', 'job_004_work_17', 'job_004_work_18', 'job_004_work_19', 'job_004_work_2', 'job_004_work_3', 'job_004_work_4', 'job_004_work_5', 'job_004_work_6', 'job_004_work_7', 'job_004_work_8', 'job_004_work_9']}


## Get Job Results

In [7]:
import logging

import requests


def get_job_results(api_url: str, job_name: str, work_id: str):
    """
    Query the results endpoint with the given job_name and work_id.

    Args:
    api_url (str): The base URL of your API Gateway.
    job_name (str): The name of the job to query.
    work_id (str): The ID of the work within the job.

    Returns:
    dict: The JSON response from the API, or None if an error occurred
    """
    api_url = api_url.rstrip("/")
    # Construct the full URL
    endpoint = f"{api_url}/results"

    # Set up the query parameters
    params = {"job_name": job_name, "work_id": work_id}

    try:
        # Make the GET request
        response = requests.get(endpoint, params=params)

        # Check the status code
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            logging.info(f"No data found for job_name={job_name} and work_id={work_id}")
            return response.json()
        else:
            response.raise_for_status()

    except requests.RequestException as e:
        logging.exception(f"An error occurred: {e}")


results = get_job_results(api_url=api_url, job_name=job_name, work_id="job_001_work_0")
print(results)

{'error': 'Results not available'}


## Update Job Results

In [ ]:
import json

import requests


def update_job_results(api_url: str, job_name: str, work_id: str):

    # Construct the full URL
    api_url = api_url.rstrip("/")
    endpoint = f"{api_url}/results"

    # Headers
    headers = {"Content-Type": "application/json"}

    updated_fields = {"work_status": "REVIEWED"}
    request_body = {"job_name": job_name, "work_id": work_id, "updated_fields": updated_fields}

    try:
        # Make the POST request
        response = requests.post(endpoint, data=json.dumps(request_body), headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            print("API Response:", data)
        else:
            print(f"Error: API request failed with status code {response.status_code}")
            print("Response:", response.text)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


update_job_results(
    api_url=api_url,
    job_name=job_name,
    work_id="job_001_work_0",
)